In [1]:
import itk
import itkwidgets
import numpy as np

In [2]:
Dimension = 3
PixelType = itk.F  # float
PointSetType = itk.PointSet[PixelType, Dimension]
MeshType = itk.Mesh[PixelType, Dimension]

In [3]:
def generate_sphere_points(radius, theta_steps, phi_steps):
    point_set = PointSetType.New()

    point_id = 0
    for theta in np.linspace(0, np.pi, theta_steps):
        for phi in np.linspace(0, 2 * np.pi, phi_steps):
            x = radius * np.sin(theta) * np.cos(phi)
            y = radius * np.sin(theta) * np.sin(phi)
            z = radius * np.cos(theta)
            itk_point = itk.Point[PixelType, Dimension]()
            itk_point.SetElement(0, x)
            itk_point.SetElement(1, y)
            itk_point.SetElement(2, z)
            point_set.SetPoint(point_id, itk_point)
            point_id += 1

    return point_set

In [4]:

def create_sphere_mesh(point_set, theta_steps, phi_steps):
    mesh = MeshType.New()

    # Copy points from PointSet to Mesh
    for i in range(point_set.GetNumberOfPoints()):
        point = point_set.GetPoint(i)
        mesh.SetPoint(i, point)

    # Define connectivity (triangular cells)
    cells_array = []

    for i in range(theta_steps - 1):
        for j in range(phi_steps):
            p1 = i * phi_steps + j
            p2 = p1 + phi_steps
            p3 = (p1 + 1) % phi_steps + i * phi_steps
            p4 = (p2 + 1) % phi_steps + (i + 1) * phi_steps

            # Create two triangles per quad on the sphere
            cells_array.append([p1, p2, p3])  # First triangle
            cells_array.append([p2, p4, p3])  # Second triangle

    # Convert cells_array to a vector container
    cells_array_np = np.array(cells_array, dtype=np.uint64)
    cells_vector = itk.vector_container_from_array(cells_array_np.flatten())

    # Set cells with triangular geometry
    mesh.SetCellsArray(cells_vector, itk.CommonEnums.CellGeometry_TRIANGLE_CELL)

    return mesh

In [5]:
# Generate a PointSet with points on a sphere
radius = 1.0
theta_steps = 10  # latitude lines
phi_steps = 20    # longitude lines
point_set = generate_sphere_points(radius, theta_steps, phi_steps)

# Create the mesh from the PointSet
mesh = create_sphere_mesh(point_set, theta_steps, phi_steps)

In [6]:
def save_mesh_as_vtk(mesh, filename):
    writer = itk.MeshFileWriter[MeshType].New()
    writer.SetFileName(filename)
    writer.SetInput(mesh)
    writer.Update()


In [7]:
# Save the mesh
save_mesh_as_vtk(mesh, 'mesh.vtk')